### Import Divvy Bike Data: 

mongoimport --type csv -d chicago_bikes -c divvy_ridedata --headerline 202201-divvy-tripdata.csv<br>
mongoimport --type csv -d chicago_bikes -c divvy_ridedata --headerline 202202-divvy-tripdata.csv<br>
mongoimport --type csv -d chicago_bikes -c divvy_ridedata --headerline 202203-divvy-tripdata.csv<br>
mongoimport --type csv -d chicago_bikes -c divvy_ridedata --headerline 202204-divvy-tripdata.csv<br>
mongoimport --type csv -d chicago_bikes -c divvy_ridedata --headerline 202205-divvy-tripdata.csv<br>
mongoimport --type csv -d chicago_bikes -c divvy_ridedata --headerline 202206-divvy-tripdata.csv<br>
mongoimport --type csv -d chicago_bikes -c divvy_ridedata --headerline 202207-divvy-tripdata.csv<br>
mongoimport --type csv -d chicago_bikes -c divvy_ridedata --headerline 202208-divvy-tripdata.csv<br>
mongoimport --type csv -d chicago_bikes -c divvy_ridedata --headerline 202209-divvy-publictripdata.csv<br>
mongoimport --type csv -d chicago_bikes -c divvy_ridedata --headerline 202210-divvy-tripdata.csv<br>
mongoimport --type csv -d chicago_bikes -c divvy_ridedata --headerline 202211-divvy-tripdata.csv<br>
mongoimport --type csv -d chicago_bikes -c divvy_ridedata --headerline  202212-divvy-tripdata.csv<br>

### Import Weather Data: 
mongoimport --type csv -d chicago_bikes -c weather_daily --headerline weather_daily.csv

In [3]:
import pymongo
from pymongo import MongoClient, UpdateOne
import json

In [4]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [19]:
# confirm that our new database was created
print(mongo.list_database_names())

['admin', 'chicago_bikes', 'classDB', 'config', 'fruits_db', 'gardenDB', 'local', 'met', 'travel_db', 'uk_food']


In [20]:
# assign the database to a variable name
db = mongo.chicago_bikes

In [21]:
# review the collections in our new database
print(db.list_collection_names())

['divvy_ridedata_test', 'weather_daily', 'divvy_ridedata']


In [22]:
# review a document in the customer_list collection
print(db.divvy_ridedata.find_one())

{'_id': ObjectId('64e4163d948aee61d19bb3bc'), 'ride_id': 'A6CF8980A652D272', 'rideable_type': 'electric_bike', 'started_at': '2022-01-10 08:41:56', 'ended_at': '2022-01-10 08:46:17', 'start_station_name': 'Glenwood Ave & Touhy Ave', 'start_station_id': 525, 'end_station_name': 'Clark St & Touhy Ave', 'end_station_id': 'RP-007', 'start_lat': 42.012763, 'start_lng': -87.6659675, 'end_lat': 42.01256011541, 'end_lng': -87.6743671152, 'member_casual': 'casual', 'started_at_date': '2022-01-10', 'started_at_time': '08:41:56', 'ended_at_date': '2022-01-10', 'ended_at_time': '08:46:17'}


In [23]:
divvy_rides = db['divvy_ridedata']
weather_daily = db['weather_daily']

In [24]:
print(divvy_rides.count_documents({}))
print(weather_daily.count_documents({}))

5667717
365


In [28]:
documents = divvy_rides.find({})
bulk_updates = []

for document in documents:
    original_value = document["started_at"]
    parts = original_value.split(" ")
    date_part = parts[0]
    time_part = parts[1]
    bulk_updates.append(
        pymongo.UpdateOne(
            {"_id": document["_id"]},
            {
                "$set": {
                    "started_at_date": date_part,
                    "started_at_time": time_part
                }
            }
        )
    )

# Execute bulk write operations
divvy_rides.bulk_write(bulk_updates)


In [29]:
documents = divvy_rides.find({})
bulk_updates = []

for document in documents:
    original_value = document["ended_at"]
    parts = original_value.split(" ")
    date_part = parts[0]
    time_part = parts[1]
    bulk_updates.append(
        pymongo.UpdateOne(
            {"_id": document["_id"]},
            {
                "$set": {
                    "ended_at_date": date_part,
                    "ended_at_time": time_part
                }
            }
        )
    )

# Execute bulk write operations
divvy_rides.bulk_write(bulk_updates)

In [26]:
print(db.divvy_ridedata.find_one())
print(db.weather_daily.find_one())

{'_id': ObjectId('64e4163d948aee61d19bb3bc'), 'ride_id': 'A6CF8980A652D272', 'rideable_type': 'electric_bike', 'started_at': '2022-01-10 08:41:56', 'ended_at': '2022-01-10 08:46:17', 'start_station_name': 'Glenwood Ave & Touhy Ave', 'start_station_id': 525, 'end_station_name': 'Clark St & Touhy Ave', 'end_station_id': 'RP-007', 'start_lat': 42.012763, 'start_lng': -87.6659675, 'end_lat': 42.01256011541, 'end_lng': -87.6743671152, 'member_casual': 'casual', 'started_at_date': '2022-01-10', 'started_at_time': '08:41:56', 'ended_at_date': '2022-01-10', 'ended_at_time': '08:46:17'}
{'_id': ObjectId('64e434c9b6c5bdee4f41aa4b'), 'date': '2022-01-02', 'cloud_cover': 90, 'precipitation': 13.81, 'min_temp': 18.81, 'max_temp': 32.77, 'morning_temp': 28.24, 'afternoon_temp': 26.64, 'evening_temp': 19.63, 'night_temp': 32.77, 'max_windspeed': 18.41}


In [41]:
divvy_rides.create_index([("started_at_date", 1)])
weather_daily.create_index([("date", 1)])


'date_1'

In [42]:
pipeline = [
    {
        '$lookup': {
            'from': 'weather_daily',
            'localField': 'started_at_date',
            'foreignField': 'date',
            'as': 'weather_data'
        }
    },
    {
        '$unwind': {
            'path': '$weather_data',
            'preserveNullAndEmptyArrays': True
        }
    },
    {
        '$merge': {
            'into': 'divvy_ridedata_merged',  # Replace with your new collection name
            'whenMatched': 'merge',  # Merge documents with matching _id fields
            'whenNotMatched': 'insert'  # Insert documents that don't match
        }
    },
]

divvy_rides.aggregate(pipeline)
print("Update completed successfully.")


Update completed successfully.


In [54]:
divvy_ridedata_merged = db['divvy_ridedata_merged']
divvy_ridedata_merged.find_one()

{'_id': ObjectId('64e4163d948aee61d19bb3bc'),
 'afternoon_temp': 9.73,
 'cloud_cover': 17,
 'end_lat': 42.01256011541,
 'end_lng': -87.6743671152,
 'end_station_id': 'RP-007',
 'end_station_name': 'Clark St & Touhy Ave',
 'ended_at': '2022-01-10 08:46:17',
 'ended_at_date': '2022-01-10',
 'ended_at_time': '08:46:17',
 'evening_temp': 14,
 'max_temp': 20.84,
 'max_windspeed': 16.35,
 'member_casual': 'casual',
 'min_temp': 9.5,
 'morning_temp': 13.69,
 'night_temp': 20.75,
 'precipitation': 0.24,
 'ride_id': 'A6CF8980A652D272',
 'rideable_type': 'electric_bike',
 'start_lat': 42.012763,
 'start_lng': -87.6659675,
 'start_station_id': 525,
 'start_station_name': 'Glenwood Ave & Touhy Ave',
 'started_at': '2022-01-10 08:41:56',
 'started_at_date': '2022-01-10',
 'started_at_time': '08:41:56',
 'weather_data': {'_id': ObjectId('64e434c9b6c5bdee4f41aa4e'),
  'date': '2022-01-10',
  'cloud_cover': 17,
  'precipitation': 0.24,
  'min_temp': 9.5,
  'max_temp': 20.84,
  'morning_temp': 13.69,
 

In [55]:
# review the collections in our new database
print(db.list_collection_names())

['weather_daily', 'divvy_ridedata_merged', 'divvy_ridedata']
